# Analysis of Brazilian Employment with PNADc Data

[intro]


## Setup

First step: clearing the working environment, and setting global options.

In [43]:
clear
set linesize 130

Loading the data with the infix command. The delimiters of each variable can be found in the [dictionary](data/PNADC_dict.xls).

In [44]:
#delim ;
    infix
    id_year 1-4
    id_tri 5-5
    id_state 6-7
    sex 95-95
    age 104-106
    race 107-107
    house_urban 33-33
    house_size 89-90
    house_type 402-402
    educ_lv 405-405
    educ_years 406-407
    ocup_njobs 151-151
    ocup_lf 409-409
    ocup_ocup 410-410
    ocup_plf 411-411
    ocup_low 413-413
    ocup_disc 414-414
    ocup_pension 423-423
    ocup_motive 461-461
    job_time 247-247
    job_group 419-420
    job_pos 421-422
    job_pay 444-452
    job_hours 462-464
    using "data\PNADC_012023.txt";
#delim cr


(473,335 observations read)


## Data Processing

Lets do a initial analysis of the data, see if there are any problems we need to deal with, before starting the actual study.

### Dataset Summary

Lets start with visualizing and summarizing the dataset as a whole.

Head, mid, and tail of dataset:

In [76]:
* To open in interactive window:
browse if inrange(_n, 1, 10) | inrange(_n, round(_N/2) - 5, round(_N/2) + 5) | inrange(_n, _N-9, _N)

* To print to the cell output:
*list if inrange(_n, 1, 5) | inrange(_n, round(_N/2) - 2, round(_N/2) + 2) | inrange(_n, _N-4, _N)

Basic descriptive statistics of all variables:

In [136]:
summarize, separator(99)


    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
     id_year |    473,335        2023           0       2023       2023
      id_tri |    473,335           1           0          1          1
    id_state |    473,335    30.89177    11.00204         11         53
         sex |    473,335    1.515713    .4997536          1          2
         age |    473,335     37.3425    22.22398          0        115
        race |    473,335    2.623795    1.433491          1          9
 house_urban |    473,335    1.266663    .4422152          1          2
  house_size |    473,335    3.502633      1.5974          1         21
  house_type |    473,335    2.197036     .563644          1          4
     educ_lv |    446,205    3.608048    1.919254          1          7
  educ_years |    446,205    8.518224    4.946553          0         16
  ocup_njobs |    200,106    1.031243    .1826358          1   

We can see that the occupation and job variables have lots of missing data. We will direct our attention to that in a moment.

Below we have extended details for each variable. We can also see the relation of the existence of missing values between the variables.

In [147]:
codebook, mv


--------------------------------------------------------------------------------
id_year                                                              (unlabeled)
--------------------------------------------------------------------------------

                  Type: Numeric (float)

                 Range: [2023,2023]                   Units: 1
         Unique values: 1                         Missing .: 0/473,335

            Tabulation: Freq.  Value
                      473,335  2023

--------------------------------------------------------------------------------
id_tri                                                               (unlabeled)
--------------------------------------------------------------------------------

                  Type: Numeric (byte)

                 Range: [1,1]                         Units: 1
         Unique values: 1                         Missing .: 0/473,335

            Tabulation: Freq.  Value
                      473,335  1

---------------

### Missing Data

Most of the missing data on occupation and job variables comes from individuals outside of the working universe (not in working age, mostly).

Note some facts about the data:

The variable "ocup_lf" denotes people in or out of the working force. Below we can see:

- The missing variables have a max age of 13 y/o
- The proportions of individuals out of the universe, in the force, or out of the force.
- For in-the-force individuals, those who are occupied or not.

In [124]:
summarize age if ocup_lf == .
tabulate ocup_lf, missing
tabulate ocup_ocup, missing



    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
         age |     83,952    6.843792    3.946286          0         13


    ocup_lf |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |    218,300       46.12       46.12
          2 |    171,083       36.14       82.26
          . |     83,952       17.74      100.00
------------+-----------------------------------
      Total |    473,335      100.00


  ocup_ocup |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |    200,106       42.28       42.28
          2 |     18,194        3.84       46.12
          . |    255,035       53.88      100.00
------------+-----------------------------------
      Total |    473,335      100.00


Then, we can state some facts about the variables:

- The $218,300$ obs. of "ocup_ocup" are only for those in the working force.
- Only the ones outside the force are queried about their potential occupation, "ocup_plf", or motive, "ocup_motive".
- Only the occupied ones present data for "ocup_pension"

We can check those assessments, and none throws an error.

In [128]:
assert ocup_ocup != . if ocup_lf == 1
assert ocup_plf != . if ocup_lf == 2
assert ocup_motive != . if ocup_lf == 2
assert ocup_pension != . if ocup_ocup == 1

Moving forward, we can see that "ocup_low" and "ocup_disc" have lots of missing data. This is because those questions are only considered for people that wanted to work more than they did.


Lastly, all the job variables are set in the universe of occupied people. The "job_pay" variable also has missing values in cases where there was no type of payment in any of the jobs.

But, the "ocup_njobs" variable doesn't have a "0 jobs" value. We must create it based on the missing individuals with working-age:

In [45]:
replace ocup_njobs = 0 if age >= 14 & missing(ocup_njobs)

(189,277 real changes made)


### Data Visualization

Now, lets visualize each variable.

#### Numeric Variables

In [ ]:
foreach var in house_size educ_years job_hours age job_pay {
    qui histogram `var' in 1/l, discrete frequency normal
    graph export "figures/`var'.png", replace
}

The figures can be seen in the [figures/](figures/) folder. We can see that there are some outliers in "job_pay". Lets get a closer look:

In [10]:
egen job_pay_bin = cut(job_pay), at(0(10000)100000 200000 3000000)
tabulate job_pay_bin


(278,422 missing values generated)


job_pay_bin |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |     34,582       17.74       17.74
      10000 |     78,717       40.39       58.13
      20000 |     33,632       17.25       75.38
      30000 |     16,841        8.64       84.02
      40000 |      7,917        4.06       88.08
      50000 |      7,084        3.63       91.72
      60000 |      3,431        1.76       93.48
      70000 |      2,143        1.10       94.58
      80000 |      1,927        0.99       95.57
      90000 |        807        0.41       95.98
     100000 |      5,853        3.00       98.98
     200000 |      1,979        1.02      100.00
------------+-----------------------------------
      Total |    194,913      100.00


This might motivate us to, in the future, drop individuals with really large payments. For now, lets just see the values without outliers:

In [11]:
qui histogram job_pay in 1/l if job_pay <= 100000, bins(40) frequency normal
graph export "figures/job_pay_trimmed.png", replace



(file figures/job_pay_trimmed.png not found)
file figures/job_pay_trimmed.png saved as PNG format


#### Categorical Variables

The individual's basic characteristics:

In [15]:
tab sex race, nowrap


           |                               race
       sex |         1          2          3          4          5          9 |     Total
-----------+------------------------------------------------------------------+----------
         1 |    88,655     22,926      1,223    115,217      1,184         25 |   229,230 
         2 |    97,757     23,182      1,574    120,226      1,324         42 |   244,105 
-----------+------------------------------------------------------------------+----------
     Total |   186,412     46,108      2,797    235,443      2,508         67 |   473,335 


The household characteristics:

In [24]:
tab house_urban house_size


house_urba |                                             house_size
         n |         1          2          3          4          5          6          7          8          9 |     Total
-----------+---------------------------------------------------------------------------------------------------+----------
         1 |    23,958     74,986     94,215     84,444     40,520     16,326      6,874      2,896      1,485 |   347,114 
         2 |     6,814     26,158     31,413     30,088     16,575      7,992      3,647      1,664        990 |   126,221 
-----------+---------------------------------------------------------------------------------------------------+----------
     Total |    30,772    101,144    125,628    114,532     57,095     24,318     10,521      4,560      2,475 |   473,335 


house_urba |                                             house_size
         n |        10         11         12         13         14         15         16         17         18 |     Tot

The number of jobs, and the filtered versions for above 16 and above 18 years old:

In [40]:
tab ocup_njobs, missing
tab ocup_njobs if age >= 16, missing
tab ocup_njobs if age >= 18, missing



 ocup_njobs |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |    189,277       39.99       39.99
          1 |    194,163       41.02       81.01
          2 |      5,634        1.19       82.20
          3 |        309        0.07       82.26
          . |     83,952       17.74      100.00
------------+-----------------------------------
      Total |    473,335      100.00


 ocup_njobs |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |    176,332       46.95       46.95
          1 |    193,333       51.47       98.42
          2 |      5,624        1.50       99.92
          3 |        309        0.08      100.00
------------+-----------------------------------
      Total |    375,598      100.00


 ocup_njobs |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |    164,366       45.50       45.50
          1 |    190,931       52.86  

The occupancy status:

In [33]:
tab ocup_lf ocup_ocup, missing
tab ocup_lf ocup_plf, missing



           |            ocup_ocup
   ocup_lf |         1          2          . |     Total
-----------+---------------------------------+----------
         1 |   200,106     18,194          0 |   218,300 
         2 |         0          0    171,083 |   171,083 
         . |         0          0     83,952 |    83,952 
-----------+---------------------------------+----------
     Total |   200,106     18,194    255,035 |   473,335 


           |             ocup_plf
   ocup_lf |         1          2          . |     Total
-----------+---------------------------------+----------
         1 |         0          0    218,300 |   218,300 
         2 |    18,910    152,173          0 |   171,083 
         . |         0          0     83,952 |    83,952 
-----------+---------------------------------+----------
     Total |    18,910    152,173    302,252 |   473,335 


The low/none occupancy specific cases:

In [39]:
tab ocup_low if job_hours < 40, missing
tab ocup_disc if ocup_njobs == 0, missing



   ocup_low |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |     11,012       21.12       21.12
          . |     41,133       78.88      100.00
------------+-----------------------------------
      Total |     52,145      100.00


  ocup_disc |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |     12,141        6.41        6.41
          . |    177,136       93.59      100.00
------------+-----------------------------------
      Total |    189,277      100.00


Pension contributors:

In [41]:
tab ocup_pension


ocup_pensio |
          n |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |    119,791       59.86       59.86
          2 |     80,315       40.14      100.00
------------+-----------------------------------
      Total |    200,106      100.00


Position at and groups of jobs:

In [52]:
tab job_pos
tab job_group



    job_pos |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |      5,832        2.91        2.91
          2 |     21,844       10.92       13.83
          3 |     14,672        7.33       21.16
          4 |     15,224        7.61       28.77
          5 |     42,885       21.43       50.20
          6 |     20,377       10.18       60.38
          7 |     25,202       12.59       72.98
          8 |     17,062        8.53       81.51
          9 |     35,376       17.68       99.18
         10 |      1,612        0.81       99.99
         11 |         20        0.01      100.00
------------+-----------------------------------
      Total |    200,106      100.00


  job_group |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |     29,862       14.92       14.92
          2 |     22,262       11.13       26.05
          3 |     13,961        6.98       33.02
          4 |     36,286    

Seniority at job:

In [53]:
tab job_time


   job_time |      Freq.     Percent        Cum.
------------+-----------------------------------
          1 |      7,693        3.84        3.84
          2 |     36,036       18.01       21.85
          3 |     20,122       10.06       31.91
          4 |    136,255       68.09      100.00
------------+-----------------------------------
      Total |    200,106      100.00


## Data Exploration